<a href="https://colab.research.google.com/github/monteroanibal/Anibal-Montero-Varios/blob/main/Clasificador_PQRs_ANT_online_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clasificación de PQRs - GIT Soluciones Informáticas - SSIT - ANT**

# Por favor, pulse en el siguiente botón PLAY. A continuación, pulse el botón *Elegir archivos*. Posteriormente, seleccione los archivos .pdf desde su ubicación local, con los radicados de PQRs que va a clasificar. 

In [ ]:
                                                                                                                                                                                     from google.colab import files; uploaded = files.upload() 

#A continuación, por favor, pulse el siguiente botón PLAY, y en unos minutos, el proceso descargará automáticamente el listado de los radicados indicando si es o no una solicitud simple.

---

Reubicar los archivos recién subidos.

In [2]:
!mkdir -p /content/archivos
%cd /content
%mv *.pdf /content/archivos/

/content


In [3]:
%%capture
!pip install Pillow==9.2.0

Luego de que se reinicie el entorno, se debe ejecutar el resto del notebook. Se ve el resultados al final del notebook y se descarga automáticamente la lista de radicados clasificados en un libro de Excel.

In [4]:
#import os
#os.kill(os.getpid(), 9)

Instalación de librerías para la conversión de pdfs a texto.
%%capture
!sudo apt install tesseract-ocr
!pip install pytesseract==0.3.9
!pip install pdf2image
!pip install datefinder
!apt-get install poppler-utils 
!apt-get update
!pip install pdfplumber
!sudo apt-get install imagemagick libmagickwand-dev
!sudo apt-get install tesseract-ocr-spa
Se debe modificar un archivo de ``pytesseract``, por lo que se realiza esta parte de código.

In [5]:
%%capture
!sudo apt install tesseract-ocr
!pip install pytesseract==0.3.9
!pip install pdf2image
!pip install datefinder
!apt-get install poppler-utils 
!apt-get update
!pip install pdfplumber
!sudo apt-get install imagemagick libmagickwand-dev
!sudo apt-get install tesseract-ocr-spa

Se debe modificar un archivo de ``pytesseract``, por lo que se realiza esta parte de código.

In [6]:
f = open("/etc/ImageMagick-6/policy.xml", "w")
f.write(
    """
    <?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE policymap [
    <!ELEMENT policymap (policy)+>
    <!ELEMENT policy (#PCDATA)>
    <!ATTLIST policy domain (delegate|coder|filter|path|resource) #IMPLIED>
    <!ATTLIST policy name CDATA #IMPLIED>
    <!ATTLIST policy rights CDATA #IMPLIED>
    <!ATTLIST policy pattern CDATA #IMPLIED>
    <!ATTLIST policy value CDATA #IMPLIED>
    ]>
    <!--
      Configure ImageMagick policies.

      Domains include system, delegate, coder, filter, path, or resource.

      Rights include none, read, write, and execute.  Use | to combine them,
      for example: "read | write" to permit read from, or write to, a path.

      Use a glob expression as a pattern.

      Suppose we do not want users to process MPEG video images:

        <policy domain="delegate" rights="none" pattern="mpeg:decode" />

      Here we do not want users reading images from HTTP:

        <policy domain="coder" rights="none" pattern="HTTP" />

      Lets prevent users from executing any image filters:

        <policy domain="filter" rights="none" pattern="*" />

      The /repository file system is restricted to read only.  We use a glob
      expression to match all paths that start with /repository:
      
        <policy domain="path" rights="read" pattern="/repository/*" />

      Let's prevent possible exploits by removing the right to use indirect reads.

        <policy domain="path" rights="none" pattern="@*" />

      Any large image is cached to disk rather than memory:

        <policy domain="resource" name="area" value="1GB"/>

      Define arguments for the memory, map, area, width, height, and disk resources
      with SI prefixes (.e.g 100MB).  In addition, resource policies are maximums
      for each instance of ImageMagick (e.g. policy memory limit 1GB, -limit 2GB
      exceeds policy maximum so memory limit is 1GB).
    -->
    <policymap>
      <!-- <policy domain="resource" name="temporary-path" value="/tmp"/> -->
      <policy domain="resource" name="memory" value="4GiB"/>
      <policy domain="resource" name="map" value="4GiB"/>
      <policy domain="resource" name="width" value="16KP"/>
      <policy domain="resource" name="height" value="16KP"/>
      <policy domain="resource" name="area" value="128MB"/>
      <policy domain="resource" name="disk" value="4GiB"/>
      <!-- <policy domain="resource" name="file" value="768"/> -->
      <!-- <policy domain="resource" name="thread" value="4"/> -->
      <!-- <policy domain="resource" name="throttle" value="0"/> -->
      <!-- <policy domain="resource" name="time" value="3600"/> -->
      <!-- <policy domain="system" name="precision" value="6"/> -->
      <!-- not needed due to the need to use explicitly by mvg: -->
      <!-- <policy domain="delegate" rights="none" pattern="MVG" /> -->
      <!-- use curl -->
      <policy domain="delegate" rights="none" pattern="URL" />
      <policy domain="delegate" rights="none" pattern="HTTPS" />
      <policy domain="delegate" rights="none" pattern="HTTP" />
      <!-- in order to avoid to get image with password text -->
      <policy domain="path" rights="none" pattern="@*"/>
      <policy domain="cache" name="shared-secret" value="passphrase" stealth="true"/>
      <!-- disable ghostscript format types -->
      <!-- <policy domain="coder" rights="none" pattern="PS" /> -->
      <policy domain="coder" rights="none" pattern="PS2" />
      <policy domain="coder" rights="none" pattern="PS3" />
      <policy domain="coder" rights="none" pattern="EPS" />
      <policy domain="coder" rights="read|write" pattern="PDF" />
      <policy domain="coder" rights="none" pattern="XPS" />
      <policy domain="coder" rights="read|write" pattern="LABEL" />
    </policymap>
    """
)
f.close()

Se cargan las librerías necesarias.

In [7]:
%%capture
#para mirar el tiempo que toma
from datetime import datetime

#para explorar directorios y nombres de archivos
import os
import glob

# para guardar y cargar los modelos ajustados de scikitlearn en un pickle
# importante usar la libreriia dill en vez de pickle para que guarde correctamente la clase que normaliza el texto
!pip install dill
from dill import dumps, loads

# para trabajar con arrays
import numpy as np

# para ver el resultado y exportarlo a excel
import pandas as pd
from google.colab import files

Librerías necesarias para la conversión de los pdfs a texto.

In [8]:
import os
import shutil
import pdfplumber
import cv2 as cv
import pytesseract
from pytesseract import Output
from time import time, ctime
from tqdm.notebook import trange, tqdm

Librerías para el pickle que se va a cargar.

In [9]:
%%capture
# para hacer remplazos de expresiones regulares
import re

# lematizador spacy español
!pip install -U spacy
!python -m spacy download es_core_news_sm
import spacy
import es_core_news_sm
parser_spacy = es_core_news_sm.load()

# tokenizador de nltk y listado stopwords en espanyol
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sp_stop_words = stopwords.words('spanish')

Se carga el modelo ajustado desde GitHub y se elimina el .pickle

In [10]:
%%capture
!wget https://github.com/monteroanibal/Anibal-Montero-Varios/raw/main/modelo_clasif_PQRs_v1.pickle

filnam = "modelo_clasif_PQRs_v1.pickle"
dilfil = open(filnam, "rb")
PL = loads(dilfil.read())
dilfil.close()

!rm -rf modelo_clasif_PQRs_v1.pickle

Se ubica la carpeta con los pdfs.

In [11]:
%%capture
%cd /content

In [12]:
orig_folder = os.listdir()
ppal_folder = orig_folder.copy()
flist = ['.config', 'sample_data', '.ipynb_checkpoints', 'AUX_IMAGES', 'OUTPUT']
for element in orig_folder:
  if element in flist or ".zip" in element:
    ppal_folder.remove(element)
ppal_folder = ppal_folder[0]
print('Nombre carpeta: -> ' + ppal_folder)

Nombre carpeta: -> archivos


Carpeta principal.

In [13]:
ROOT_FOLDER = os.getcwd()
print(ROOT_FOLDER)

/content


Carpeta para los archivos de texto generados.

In [14]:
TXT_FOLDER = os.sep.join([ROOT_FOLDER,"OUTPUT"])
if not os.path.exists(TXT_FOLDER):
    os.mkdir(TXT_FOLDER)
print(TXT_FOLDER)

/content/OUTPUT


Carpeta para los archivos de imagen que fueron usados:

In [15]:
IMAGE_FOLDER = os.sep.join([ROOT_FOLDER,"AUX_IMAGES"])
if not os.path.exists(IMAGE_FOLDER):
    os.mkdir(IMAGE_FOLDER)
print(IMAGE_FOLDER)

/content/AUX_IMAGES


Funciones necesarias para la conversión de los pdfs a texto.

In [16]:
def verify_folder(dir_folder):
  "Crea una carpeta para guardar las imagenes obtenidas"
  #dir_folder = os.sep.join([ROOT_FOLDER, dir_folder])
  if not os.path.exists(dir_folder):
    os.mkdir(dir_folder)
  return dir_folder

In [17]:
def validate_content(text):
  "Valida si una cadena es suficiente para considerarse texto"
  photo_foot = [
      "Scanned by TapScanner"
  ]
  invalid_options = [
      "  ",
      "   "
  ]
  
  photo = False
  for option in photo_foot:
    if text.__contains__(option):
      photo = True
      text = ""
      break

  
  if not photo:
    for option in (invalid_options+photo_foot):
      text = text.replace(option, "")
  
  return text, photo

In [18]:
def image_text(imgFile, ncuadro = 23, par_c = 5, preprocess=False):
  """
  Obtiene texto de una imagen binarizada y sin binarizar
  Input: Ruta de ubicacion de la imagen (string)
  """
  print(f"----------Imagen: {imgFile}")
  img = cv.imread(imgFile)
  
  if preprocess:
    ad_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    th2 = cv.adaptiveThreshold(ad_img,255,cv.ADAPTIVE_THRESH_MEAN_C,\
                cv.THRESH_BINARY,ncuadro,par_c)
    txt2 = pytesseract.image_to_string(th2)#, lang="spa")
    return txt2
  else:
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    txt1 = pytesseract.image_to_string(img)#, lang="spa")
    
    return txt1

In [19]:
def process_file(filename):
  """
    Función principal, que se encarga de procesar los 
    archivos PDF ingresados.
  """
  head = os.path.split(filename)[1].replace('.pdf','')
  print(head)
  folder_output = verify_folder(os.sep.join([IMAGE_FOLDER, head]))
  #Se crea el TXT para incluir lineas de texto
  f = open(os.sep.join([TXT_FOLDER, head + ".txt"]), "w")
  # Abre el PDF para procesarlo
  archivo = pdfplumber.open(filename)
  pags = {}
  k=1
  # Lee cada pagina del PDF
  for page in tqdm(archivo.pages):
    print(f"-----------------PAGINA {k}---------------")
    t0 = time()
    # Extrae el texto de la pagina
    text_page = page.extract_text()
    # Preprocesa el texto 
    text_page = validate_content(text_page)
    photo = text_page[1]
    if len(text_page[0])>0:
      # Caso de texto digitalizado
      # Guarda el texto en un diccionario
      pags[k]= text_page[0]
      #print(text_page[0])
    else:
      # Caso como imagen
      ## Guarda la imagen
      image_file = os.sep.join([IMAGE_FOLDER, head, f"{k}.jpg"])
      page.to_image(resolution=200).save(image_file, format="JPEG")
      # Procesa la imagen
      ## Verifica tipo de imagen

      ## Caso en el que es digitalizado

      ## Caso en el que es un escaneo

      if photo:
        ## Caso en el que es foto
        ff = image_text(image_file, preprocess=True)
      else:
        ff = image_text(image_file)
      #print("Normal", ff)
      #Guarda el dato en el diccionario
      pags[k]= ff
    
    #Registro de tiempo
    t2 = time()
    print(f"Tiempo: {t2-t0}")

    #Guarda la linea en el TXT
    f.write(f"\n----------------PAGINA {k}---------------\n")
    f.write(pags[k])
    k = k+1
  f.close()

Conversión de los pdfs a texto.

In [20]:
%%capture
folder_pdf = os.sep.join([os.getcwd(),ppal_folder])
list_files = glob.glob(os.path.join(folder_pdf,"*.pdf"))
for fil in tqdm(list_files):
  process_file(fil)

Se copian los textos en una lista.

In [21]:
dirs = glob.glob(os.sep.join([ROOT_FOLDER,"OUTPUT/*.txt"]))
corpus_pr = []

for ruta in dirs:
  with open(ruta) as f:
    corpus_pr.append( f.read() )

Predicciones del pipeline.

In [22]:
predictions = PL.predict(corpus_pr)

Resultado.

In [23]:
resultado = pd.DataFrame(list(zip([os.path.splitext(os.path.basename(d))[0] for d in dirs],predictions)), columns=['Num_radicado','Clasificación'])
resultado

,Num_radicado,Clasificación
0,20236200013562,Solicitud Simple
1,20236200012492,Solicitud Simple
2,20236200011642,Solicitud Simple


Descarga automática.

In [24]:
namres = 'Clasificacion_Radicados' + datetime.now().strftime('_%m%d%Y_%H%M%S') + '.xlsx'
resultado.to_excel(namres, sheet_name='Clasificacion', index=False)
files.download(namres)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---